In [1]:

# crawl outward to build a keyword list for fmri papers

In [1]:
import gensim
import spacy

import csv
import time
import numpy as np

import mysql.connector as mysql

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import Row, StructType, StructField, IntegerType, StringType

import os
import time

/home/brendanchambers/.conda/envs/embedding-base/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [6]:
run_locally = False # set this switch for RCC vs laptop mode

if run_locally:
    path2bin = '~/Dropbox/Data/w2v/'
    pubmed_w2v = path2bin + 'PubMed-and-PMC-w2v.bin'
else:
    path2bin = '/project2/jevans/brendan/w2v/pretrained/'
    #pubmed_w2v = path2bin + 'PubMed-w2v.bin'
    #pubmed_w2v = path2bin + 'PubMed-and-PMC-w2v.bin'
    pubmed_w2v = path2bin + 'wikipedia-pubmed-and-PMC-w2v.bin'
    
    

In [8]:
# load the pretrained word vectors (trained on biomedical data)
model = gensim.models.KeyedVectors.load_word2vec_format(
    pubmed_w2v, binary=True)

# prepare a tokenizer to process the abstracts

nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load("en_core_sci_sm")  # load scispacy
#tokenizer = nlp.Defaults.create_tokenizer(nlp)  # ideally would want to use the scispacy version


In [9]:

words2try = ['fmri','bold']
keywords = []
for word in words2try:
    
    try:
        v = model.word_vec(word)
        keywords.append(word)
    except Exception as e:
        print(e)
    
print(keywords)
    



# get dimensionality
D = np.shape(model.word_vec('cell'))[0]
print(D)


# find neighbors



['fmri']
200


In [6]:
# load the pubmed word2vec binary file

# check for the entry 'fmri'

# find neighbors



In [2]:
# load the papers using pyspark

# filter on the text
'''
SUBMIT_ARGS = "--driver-class-path file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar --jars file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

db_name = 'test_pubmed'  # db name collisons? https://stackoverflow.com/questions/14011968/user-cant-access-a-database
url = "jdbc:mysql://localhost:3306/{}?useUnicode=true&useJDBCCompliantTimezoneShift=true&useLegacyDatetimeCode=false&serverTimezone=America/Chicago".format(db_name)  # mysql runs on port 3306
client_config = {'unix_socket':'/home/brendanchambers/.sql.sock',
                'database': db_name}  # for python connector

print('initializing spark')
# init spark
conf = SparkConf()
conf = (conf.setMaster('local[*]')
       .set('spark.driver.memory','26G')
       .set("spark.jars", "/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar"))        

sc = SparkContext(conf=conf)
#sc.addJar('home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar')  # temp
spark = SparkSession(sc)  # don't need this for vanilla RDDs

print(sc._conf.getAll())
'''

# for mysql connector
client_config = {'unix_socket':'/home/brendanchambers/.sql.sock',
                            'database':'test_pubmed'}
                            #'use_pure':True}

initializing spark
[('spark.repl.local.jars', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.driver.memory', '26G'), ('spark.driver.port', '34140'), ('spark.driver.host', 'midway2-0131.rcc.local'), ('spark.jars', '/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.executor.id', 'driver'), ('spark.app.name', 'pyspark-shell'), ('spark.app.id', 'local-1569536401180'), ('spark.rdd.compress', 'True'), ('spark.driver.extraClassPath', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]


In [8]:
# try filtering first (this isn't working - sql syntax)

In [17]:

# step 2 build a list of papers

# get all papers post-1985
cutoffYear = 1985
keyword = 'fmri'

start_time = time.time()

db = mysql.connect(**client_config)
sql = '''SELECT A.pmid, A.title, A.abstract, M.year, M.journal
            FROM metadata as M
            JOIN abstracts as A
            ON M.pmid = A.pmid
            WHERE 
              (M.year > {}) AND
              (A.abstract LIKE '%{}%')'''.format(cutoffYear, keyword)

cursor = db.cursor(buffered=False)
cursor.execute(sql)

subset = []
for i,row in enumerate(cursor):
    
    abstract = row[2]
    
    if keyword in abstract:
        print(abstract)
        
    subset.append(row)
    
    if i > 1:
        break

cursor.close()
db.commit()

end_time = time.time()
print("journals enumerated in {} s".format(end_time - start_time))


# filter on keywords


InternalError: Unread result found

In [10]:
# try grabbing all and then filtering

In [5]:
'''
import pyspark.sql.functions as psf
keyword='fmri'


sql = "(SELECT * FROM abstracts) AS t"
print(sql)

start_time = time.time()
df = spark.read.format('jdbc').option("url", url)\
                              .option("dbtable", sql)\
                              .load().repartition(10)
df.filter(psf.lower(df.abstract).rlike(keyword))

# test
test = df.take(1)

end_time = time.time()
print("dataframe loaded in {} s".format(end_time - start_time))

print(df.rdd.getNumPartitions())
'''

'\nimport pyspark.sql.functions as psf\nkeyword=\'fmri\'\n\n\nsql = "(SELECT * FROM abstracts) AS t"\nprint(sql)\n\nstart_time = time.time()\ndf = spark.read.format(\'jdbc\').option("url", url)                              .option("dbtable", sql)                              .load().repartition(10)\ndf.filter(psf.lower(df.abstract).rlike(keyword))\n\n# test\ntest = df.take(1)\n\nend_time = time.time()\nprint("dataframe loaded in {} s".format(end_time - start_time))\n\nprint(df.rdd.getNumPartitions())\n'

In [ ]:
# how many articles in this keyword-defined core?
# save output to csv file, db table, or json


In [ ]:
# step 3  build the neighborhood of cited and citing work




In [ ]:
# step 4 map retractions into the network, and score nodes based on closeness

In [ ]:
sc.stop()